In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
import re
from tensorflow.keras.layers import LeakyReLU
# tf.config.experimental.set_memory_growth()
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#         tf.config.experimental.set_memory_growth(gpu, True)
# tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3096)])
# # tf.config.experimental.set_memory_growth()

In [3]:
tf.__version__

'2.1.0'

## Load data

In [4]:
data = pd.read_csv('data/cards.csv')    
prices = pd.read_csv('data/prices.csv')

pd.options.display.max_columns = data.shape[1] #display option
# data.describe(include='all')

C:\Users\Przemyslaw\.conda\envs\mtg\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,9,12,16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
prices_no_duplicates = prices.drop_duplicates(subset=['uuid'])
# prices_no_duplicates.describe(include='all')

In [6]:
# prices.head()
# data.head()
data.columns

Index(['index', 'id', 'artist', 'asciiName', 'borderColor', 'colorIdentity',
       'colorIndicator', 'colors', 'convertedManaCost', 'duelDeck',
       'edhrecRank', 'faceConvertedManaCost', 'flavorName', 'flavorText',
       'frameEffect', 'frameEffects', 'frameVersion', 'hand', 'hasFoil',
       'hasNoDeckLimit', 'hasNonFoil', 'isAlternative', 'isArena', 'isBuyABox',
       'isDateStamped', 'isFullArt', 'isMtgo', 'isOnlineOnly', 'isOversized',
       'isPaper', 'isPromo', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'layout',
       'leadershipSkills', 'life', 'loyalty', 'manaCost', 'mcmId', 'mcmMetaId',
       'mtgArenaId', 'mtgoFoilId', 'mtgoId', 'multiverseId', 'name', 'names',
       'number', 'originalText', 'originalType', 'otherFaceIds', 'power',
       'printings', 'purchaseUrls', 'rarity', 'scryfallId',
       'scryfallIllustrationId', 'scryfallOracleId', 'setCode', 'side',
       'subtypes', 'supertypes', 'tcgplayerProduc

In [7]:
data_merged = pd.merge(data, prices_no_duplicates, on='uuid', how='inner')

In [8]:
data_merged.drop_duplicates(subset='name', keep="last", inplace=True)

### Include selected features

In [9]:
# len(data_merged)
data_merged = data_merged[['name', 'text', 'price', 'convertedManaCost', 'rarity', 'power', 'life', 'colors', 'manaCost', 'types', 'subtypes']]
# print(data_merged.dtypes)

In [10]:
data_merged[data_merged.price > 100][['name', 'text', 'price', 'convertedManaCost']]

,name,text,price,convertedManaCost
5003,Ali from Cairo,Damage that would reduce your life total to le...,304.95,4.0
5006,Bazaar of Baghdad,"{T}: Draw two cards, then discard three cards.",1499.99,0.0
5012,City in a Bottle,Whenever one or more other nontoken permanents...,399.95,2.0
5021,Diamond Valley,"{T}, Sacrifice a creature: You gain life equal...",250.99,0.0
5022,Drop of Honey,"At the beginning of your upkeep, destroy the c...",419.96,1.0
...,...,...,...,...
34102,Ravages of War,Destroy all lands.,179.92,4.0
34182,Zodiac Dragon,When Zodiac Dragon is put into your graveyard ...,243.20,9.0
35944,Grim Tutor,Search your library for a card and put that ca...,231.87,3.0
39121,"Ugin, the Spirit Dragon","[+2]: Ugin, the Spirit Dragon deals 3 damage t...",138.21,8.0


In [11]:
data_merged['price'].quantile(0.95)

6.6

In [12]:
data_merged['price'].describe(percentiles=[0.5, 0.7, 0.8, 0.9, 0.95])

count    20652.000000
mean         2.995274
std         39.146253
min          0.010000
50%          0.040000
70%          0.160000
80%          0.480000
90%          2.310000
95%          6.600000
max       2999.990000
Name: price, dtype: float64

In [13]:
data_merged = data_merged[data_merged.price < data_merged['price'].quantile(0.95)]

In [14]:
data_merged['price'].describe()

count    19618.000000
mean         0.386039
std          0.970452
min          0.010000
25%          0.030000
50%          0.040000
75%          0.180000
max          6.550000
Name: price, dtype: float64

### Zamiania typu na string dla tokenizera

In [15]:
data_merged['text'] = data_merged['text'].astype(str)

### Tokenizacja

In [16]:
# vocab_size = 1000 #cannot be too long for lstm layer
vocab_size = 4096
max_length = 287 #95% cards will fit this text length
# max_length = 196 #80%

# max_length = data_merged.text.map(lambda x: len(x)).max()
# max_length = int((max_length * 3) / 4)

# max_length = 100
trunc_type='post'
# padding_type='post'
padding_type = 'pre'

In [17]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(data_merged['text'])
sequences = tokenizer.texts_to_sequences(data_merged['text'])

In [18]:
tokenizer.word_index

{'<OOV>': 1,
 'the': 2,
 'creature': 3,
 'you': 4,
 'a': 5,
 'of': 6,
 '1': 7,
 'your': 8,
 'target': 9,
 'to': 10,
 'this': 11,
 'it': 12,
 'card': 13,
 'turn': 14,
 '2': 15,
 'control': 16,
 'battlefield': 17,
 'or': 18,
 'and': 19,
 'that': 20,
 'if': 21,
 'damage': 22,
 'end': 23,
 'may': 24,
 'on': 25,
 'until': 26,
 'put': 27,
 'each': 28,
 'with': 29,
 'player': 30,
 'enters': 31,
 'as': 32,
 'when': 33,
 'whenever': 34,
 'from': 35,
 'creatures': 36,
 'cast': 37,
 't': 38,
 'for': 39,
 'library': 40,
 'its': 41,
 'flying': 42,
 'spell': 43,
 'an': 44,
 'cards': 45,
 'deals': 46,
 'hand': 47,
 'counter': 48,
 'gets': 49,
 '3': 50,
 'graveyard': 51,
 'at': 52,
 'any': 53,
 'life': 54,
 'sacrifice': 55,
 "can't": 56,
 'cost': 57,
 'be': 58,
 'x': 59,
 'is': 60,
 'draw': 61,
 'beginning': 62,
 'mana': 63,
 'then': 64,
 'in': 65,
 'exile': 66,
 'two': 67,
 'b': 68,
 'all': 69,
 'into': 70,
 'return': 71,
 'r': 72,
 'their': 73,
 'opponent': 74,
 'gain': 75,
 'land': 76,
 'upkeep': 7

### Padding

In [19]:
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
# padded

## Nontext features

In [20]:
max_nontext_length = 0

### Attach converted mana cost

In [21]:
# # data_merged['convertedManaCost'].to_numpy().reshape(-1, 1)
# padded = np.append(padded, data_merged['convertedManaCost'].to_numpy().reshape(-1, 1), axis=1)
# max_nontext_length += 1

### get rarity dummies

In [22]:
padded = np.append(padded, pd.get_dummies(data_merged['rarity'], drop_first=True).to_numpy(), axis=1)
max_nontext_length += 3

### Power

In [23]:
data_merged['power'] = pd.to_numeric(data_merged['power'], errors='coerce')
data_merged['power'].fillna(data_merged['power'].mean(), inplace=True)
# data_merged['power'].fillna(0, inplace=True)
padded = np.append(padded, data_merged['power'].to_numpy().reshape(-1, 1), axis=1)
max_nontext_length += 1
# data_merged['power']

### Life

In [24]:
data_merged['life'].fillna(data_merged['life'].mean(), inplace=True)
# data_merged['life'].fillna(0, inplace=True)
padded = np.append(padded, data_merged['life'].to_numpy().reshape(-1, 1), axis=1)
max_nontext_length += 1
# data_merged['life']

### Mana cost

In [25]:
data_merged['manaCost'] = data_merged['manaCost'].astype(str)
index = data_merged['manaCost'].apply(lambda x: [y for y in [m.start() for m in re.finditer('{+|}+', x)]])
# index

In [26]:
colors = ['R', 'G', 'W', 'B', 'U']
colors_combinations = ['R/W', 'R/G', 'B/R', 'U/R', 'G/W', 'W/B', 'W/U', 'B/G', 'G/U', 'U/B']
life_pay_combinations = ['R/P', 'W/P', 'G/P', 'B/P', 'U/P']

combined = colors + colors_combinations + life_pay_combinations

manacost = np.zeros((len(data_merged['manaCost']), 22)) #first 2 for X pay and colorless pay
k = 0

for i, row in data_merged['manaCost'].iteritems():
#     print(i)
    values = []
    for j in range(0, len(index[i]), 2):
        first = index[i][j]
        second = index[i][j + 1]
        values.append(row[first + 1:second])
    for val in values:
        if val.isnumeric():
            manacost[k, 20] = int(val)
        elif val == 'X':
            manacost[k, 21] = 1
        else:
            try:
                occ = combined.index(val)
                manacost[k, occ] += 1
            except ValueError:
                pass
    k+= 1

padded = np.append(padded, manacost, axis=1)
max_nontext_length += 22

### types

In [27]:
types_dummies = pd.get_dummies(data_merged['types'], drop_first=True)
types_len = len(types_dummies.iloc[0, :])

padded = np.append(padded, types_dummies.to_numpy(), axis=1)
max_nontext_length += types_len

### subtypes

In [28]:
for i, row in data_merged['subtypes'].iteritems():
    print(row)

nan
Drone
Construct
Spider
nan
nan
Human,Wizard
nan
nan
Golem
nan
nan
Zombie,Beast
nan
Bringer
Bringer
Bringer
Bringer
Bringer
nan
nan
nan
Golem
nan
Beast
Elemental
nan
Aura
nan
Zombie,Crocodile
nan
Drake
nan
Equipment
nan
nan
Beast
nan
nan
Insect
Human,Warrior
nan
nan
Goblin,Warrior
nan
Equipment
nan
nan
Equipment
nan
Equipment
nan
nan
nan
Hellion,Beast
Elf,Druid
Goblin,Artificer
Ogre
nan
Cat,Soldier
nan
Elephant,Cleric
Elephant,Soldier
Golem
nan
Vampire
Human,Rogue,Rigger
Golem
Myr
Equipment
Zombie
Equipment
Ouphe,Rogue
Elemental
nan
nan
nan
Wall
nan
nan
nan
nan
nan
Construct
nan
nan
Construct
Equipment
nan
Insect
Aura
nan
Egg
nan
Construct
Snake
nan
Troll,Shaman
Construct
Dinosaur,Beast
nan
Vedalken,Wizard
nan
Elf,Shaman
Elf,Warrior,Scout
Cat,Knight
Human,Wizard
Construct
Aura
Human,Pilot
Human,Artificer
Elf,Artificer,Druid
Dwarf,Artificer
Human,Artificer
Vedalken,Artificer
Cat
Dwarf,Scout
nan
Elf,Soldier
Ajani
Dwarf,Rogue
Construct
Juggernaut
Assembly-Worker
nan
Construct
nan
Vehic

Specter
nan
Rhino,Soldier
nan
nan
nan
nan
Leviathan
nan
Vampire
Aura
nan
Human,Knight
Avatar
Human,Wizard
Demon
nan
Human,Advisor
nan
Elf,Shaman
Human,Rogue
nan
nan
Bird
Zombie,Cleric
Human,Cleric
Human,Cleric
nan
Human,Warrior
nan
nan
Elf,Soldier
Wolf
nan
Human,Advisor
Goblin,Rogue
Goblin
Vampire
nan
nan
nan
nan
Bird,Illusion
nan
Human,Rogue,Mutant
Human,Knight
Elf,Rogue
Ooze
nan
Human,Warrior
nan
Human,Knight
Elemental
Human,Soldier
Giant,Soldier
Elemental,Cat
Dinosaur
nan
nan
Rat,Horror
Dragon
Human,Pirate
nan
Aura
Gargoyle,Illusion
Ooze
nan
nan
Human,Soldier
nan
Elemental,Chimera
nan
Bird
Human,Wizard
nan
Construct
Construct
nan
Angel
nan
nan
Human,Advisor
Construct
nan
Construct
Construct
Bird,Construct
Hound,Construct
Giant,Soldier
Human,Cleric
Spirit,Cleric
Shapeshifter
Construct
nan
Zombie
Human,Warrior
Human,Artificer
Goblin,Rogue
nan
Goblin,Rogue
Zombie,Wizard
Ogre,Shaman
Hydra
Goblin,Warrior
nan
nan
Construct
Construct
nan
Human,Rogue
Human,Rogue
Human,Rogue
Human,Wizard
nan

Spirit
Equipment
Werewolf,Horror
Vampire,Rogue
Eldrazi,Crab
Werewolf,Horror
Zombie
nan
Human,Shaman
nan
nan
Human,Scout
Ouphe
Golem
Scarecrow
Elemental,Shaman
Kithkin,Soldier
nan
nan
Elemental
Spirit
Merfolk,Rogue
nan
nan
nan
nan
Aura
Elemental
Shapeshifter
Plant,Zombie
nan
Horror
Spirit,Avatar
Hag
nan
nan
Faerie,Rogue
Dwarf,Soldier
Dwarf,Warrior
Dwarf,Soldier
Aura
nan
nan
Elemental,Spirit
nan
Scarecrow
Aura
nan
nan
Aura
Ouphe
nan
Beast
Faerie
Hag,Shaman
nan
Hag
Goblin,Warrior
Elemental
Elemental,Warrior
nan
Goblin,Knight
Scarecrow
Giant,Warrior
nan
Giant,Warrior
Faerie,Wizard
nan
nan
nan
Scarecrow
Kithkin,Wizard
Kithkin,Cleric
Equipment
nan
Spirit
Bird
Giant,Warrior
Merfolk,Rogue
Merfolk,Wizard
Horror
nan
nan
nan
Horror
Specter
nan
Shapeshifter
Hag
Noggle,Rogue
Noggle,Rogue
Noggle,Wizard
Elemental
Troll
nan
Goblin,Shaman
Kithkin,Soldier
Elemental
nan
nan
Beast
Merfolk,Wizard
Aura
Elemental
nan
Troll
nan
Shapeshifter
nan
Treefolk,Shaman
nan
Scarecrow
Aura
Scarecrow
Shapeshifter
Element

Mole
nan
Wolf,Bear
Human,Wizard
Fox
Cat,Ape
nan
nan
nan
Fox
Antelope,Lizard
Aura
nan
Human,Warrior
Cat,Beast
nan
nan
Cat
Beast
Human,Soldier
nan
Elemental,Jellyfish
Nightmare,Pangolin
Spider
nan
Nightmare
nan
nan
nan
nan
Nightmare,Squirrel
Elephant
Human,Warrior
Human,Druid
Nightmare
Cat
Dinosaur
nan
Nightmare
nan
Human,Wizard
Human,Cleric
nan
nan
Elemental,Serpent
nan
Lizard,Beast
Insect
Human,Assassin
Unicorn
Cat
nan
nan
Beast
Dinosaur
nan
Beast
nan
Egg
Aura
nan
nan
Cat,Nightmare
nan
Human,Soldier
nan
nan
Elemental,Beast
Cat
Human,Soldier
Elemental,Whale
Frog
Bird,Beast
Shark,Beast
Monkey
nan
Human,Warrior
Elemental,Dinosaur
nan
nan
nan
Dinosaur,Cat
nan
Lizard
nan
nan
Rhino,Beast
nan
Cat
Elemental,Cat
Octopus
Scorpion
nan
nan
Human,Druid
nan
Human,Soldier
Aura
Human,Warrior
Wolverine
Elk,Unicorn
nan
nan
Faerie,Dragon
nan
Bird,Goat
nan
nan
nan
nan
nan
Otter
nan
Dinosaur,Beast
Beast
nan
nan
nan
Human,Soldier
Nightmare,Horror
Shark
Fox,Bird
nan
Human,Soldier
nan
nan
Dinosaur
Human,Wizar

Elk
Angel
nan
nan
nan
nan
Illusion
Elemental
Jackal
nan
Beast
Bat
Human,Knight
nan
Equipment
Human,Shaman
nan
Cat
Fungus,Snake
nan
nan
Rhino,Soldier
nan
Serpent
Aura
nan
Sphinx
Giant
Human,Cleric
Human,Soldier
Viashino,Warrior
Skeleton,Wall
Wall
Aura
Demon
nan
Faerie
nan
Angel
Elemental,Ox
Human,Druid
Drake
Human,Berserker
Human,Wizard
nan
Goblin,Berserker
nan
nan
Vampire
Chandra
Human,Knight
Sphinx
Cyclops,Warrior
nan
Demon
nan
Aura
Elemental
nan
nan
Aura
nan
Spirit
nan
nan
nan
nan
nan
Phoenix
nan
Merfolk,Noble
nan
nan
Aura
Horror
Beast
Human,Soldier
Illusion,Beast
Aura
Hydra
nan
Rat
Zombie
Human,Monk
nan
Bird
nan
Equipment
nan
Aura
Archon
Vampire,Wizard
Equipment
Human,Knight
nan
Construct
Human,Wizard
Human,Wizard
Siren
Salamander,Rogue
nan
Human,Archer
Archon
Horse
Griffin
nan
Bird,Soldier
Human,Soldier
Vampire
Giant
nan
Wurm
Zombie
Drake
Human,Shaman
nan
Golem
Troll
nan
Insect
nan
nan
nan
Shade
nan
Human,Soldier
Dragon
Aura
Dragon
Beast
Human,Soldier
Goblin,Warrior
Goblin
nan
Huma

...

Dragon
nan
Orc,Berserker
Dragon
Naga,Wizard
Human,Soldier
Human,Assassin
nan
Djinn,Monk
Dragon
Human,Warrior
Dragon
nan
Orc,Warrior
Giant
Wolf
Devil,Horror
Vampire,Cleric
nan
nan
nan
nan
Eldrazi,Boar
Eldrazi,Insect
Insect,Horror
Eldrazi,Octopus
nan
Human,Horror
Eldrazi,Horror
Human,Wizard
nan
Human,Soldier
nan
Human,Knight
Shapeshifter
Devil
Aura
Wolf,Construct
Spirit
nan
Spirit
Zombie
nan
Spirit
nan
Human,Cleric
nan
Wolf,Spirit
Equipment
Vampire,Horror
Vampire,Horror
nan
Human,Soldier
Human,Werewolf
Treefolk
nan
Vampire,Horror
Human,Horror
Human,Warrior
nan
Demon
Orc,Warrior
Human,Warrior
Human,Warrior
Dragon
Human,Warrior
Phoenix
Dragon
Human,Warrior
Human,Warrior
Dragon
nan
Djinn,Monk
Elephant
Orc,Shaman
nan
Human,Monk
Dragon
Demon
nan
Human,Shaman
Elemental
Human
Human,Warrior
nan
Zombie,Wizard
nan
Kithkin,Soldier
Dwarf,Shaman
Faerie,Rogue
Merfolk,Wizard
Beast
nan
Angel
Elf,Druid
Vampire,Assassin
Human,Soldier
nan
nan
nan
nan
nan
nan
Nightmare,Demon
Elf,Cleric
Vampire,Assassin
nan


Human,Soldier,Archer
Human,Soldier,Scout
Human,Soldier,Rogue
Human,Soldier
Human,Soldier
Human,Soldier
Human,Soldier
Human,Soldier
Hound
Goat
Monkey
Ox
Rabbit
Bird
Snake
nan
Goblin,Shaman
Dragon
nan
Merfolk,Wizard
Human,Druid,Knight
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Human,Barbarian
Elemental
Cat,Warrior
Aura
nan
nan
nan
Human,Wizard
Vampire
nan
Aura,Curse
nan
Human,Artificer
Myr
nan
Elf,Scout
Spirit
Imp
Aura,Curse
nan
Vampire,Rogue
Lizard
Beast
Human,Soldier
Human,Wizard
Wurm
Human,Barbarian
nan
Sphinx
Spirit
Archon
nan
nan
nan
nan
nan
nan
Human,Wizard
Goblin,Soldier
nan
Human,Soldier
nan
nan
Elemental
Aura
nan
Human,Cleric
Zombie,Wolf
Spirit
Centaur,Warrior
Sphinx
nan
nan
Dryad
Human,Wizard
Elf,Druid,Warrior
nan
Aura
nan
nan
Boar,Beast
nan
nan
Human,Soldier
Human,Soldier
Aura
nan
nan
nan
nan
Bird
nan
nan
Human,Wizard
nan
nan
nan
nan
Spirit
Shapeshifter
Skeleton
Spirit
nan
nan
nan
Griffin
nan
nan
nan
Centaur,Shaman
Drake
Aura
Spirit
Cam

Fungus
nan
Spirit
Slug,Horror
Equipment
nan
nan
nan
Nahiri
Human,Cleric
Equipment
Drake
Spirit
nan
Human,Rogue
Human,Soldier
Vampire,Soldier
Vampire,Knight
nan
nan
Wolf,Spirit
Spirit
Human,Soldier
nan
nan
Human,Cleric
nan
nan
nan
Zombie
nan
Elemental,Hound
Wolf
Zombie,Rat
Spirit
Vampire,Berserker
Angel
nan
Zombie
nan
Zombie
Construct
nan
Human,Shaman,Werewolf
Vampire,Wizard
Wolf,Horror
Zombie
nan
nan
Aura
nan
Equipment
Angel
Turtle
Spirit
Wolf,Warrior
nan
Devil
nan
Equipment
Aura
Equipment
Human,Warrior,Werewolf
Sorin
Wurm
Spirit
Aura
Nightmare,Horse
nan
nan
Human,Soldier
Zombie,Horror
Zombie,Horror
Human
Spirit
nan
Vampire,Soldier
nan
nan
nan
nan
Human,Soldier
Frog,Horror
Horror
Wolf
Gargoyle
Human,Soldier
nan
Human,Scout
nan
Human,Rogue
Spirit
nan
Human
nan
nan
nan
Equipment
Wolf
Hydra
nan
nan
Spirit
Human
Vampire,Noble
nan
nan
nan
nan
nan
Human,Soldier
Human,Werewolf
Vampire,Warrior
nan
Spider
Aura
nan
nan
Scarecrow
Scarecrow
Elemental,Wolf
Spirit
nan
Goblin,Shaman
Goblin,Warrior
Sp

nan
Aura
Human,Wizard
Goat,Beast
nan
Aura
Rat
nan
Aura
Elf
Elf,Shaman
Crocodile
nan
nan
Juggernaut
Human,Cleric
nan
nan
Aura
Human,Cleric
nan
Bird
nan
nan
Elf
Goblin,Berserker
nan
Goblin
Goblin,Warrior
Goblin
Insect
Elf,Druid
Ogre
Bird,Beast
Aura
nan
Aura
Elf,Wizard
Human,Wizard
Bird
Human,Barbarian
Human,Rogue
Bird
nan
nan
nan
Insect
Aura
Insect
Aura
Human,Cleric
nan
Metathran,Soldier
Metathran,Soldier
Aura
nan
Human,Wizard
nan
Skeleton
Horror
Zombie,Hound
Spider
nan
nan
Aura
nan
nan
Human,Wizard
nan
Human,Monk,Cleric
nan
nan
nan
Elf,Druid
nan
nan
nan
nan
nan
nan
nan
nan
Angel
Aura
Horror
Imp
nan
nan
nan
nan
Horror
nan
Elf
Human,Wizard
Human,Wizard
Griffin
Bird
Elemental
nan
Golem
Angel
Aura
nan
Aura
Angel
Angel
nan
Wall
Goat,Beast
Human,Druid
Human,Druid
nan
Hound,Scout
Beast
nan
Aura
nan
nan
nan
Equipment
Golem
Goblin,Rogue
Human,Monk
nan
Human,Monk
nan
Human,Assassin
nan
nan
Zombie,Warrior
Construct
Bird,Soldier
nan
nan
nan
The,Biggest,,Baddest,,Nastiest,
nan
nan
nan
nan
Aura
nan
n

Human,Cleric
Spike
Aura
Thrull
nan
nan
Elemental
Plant,Wall
Plant,Wall
nan
nan
nan
nan
nan
Dragon
nan
nan
nan
nan
nan
nan
Golem
Elf,Druid
nan
nan
nan
nan
nan
Cat
Rat
nan
nan
nan
Human,Knight
nan
nan
Treefolk
nan
Aura
nan
nan
Elemental
Horror
nan
nan
Dragon
nan
nan
nan
Aura
Wolf
Human,Soldier
nan
nan
nan
Jellyfish
Elemental
Zombie
Human,Soldier
Human,Knight
Human,Cleric
Aura
Cyclops
nan
Elemental,Spirit
nan
Zombie
Aura
nan
nan
nan
nan
nan
nan
Giant
Wall
Bird
Djinn
Aura
nan
nan
nan
nan
nan
nan
nan
Bird
Dwarf,Berserker
Dwarf,Shaman
Aura
Illusion
Wurm
nan
nan
nan
nan
Aura
nan
nan
Djinn
nan
Elemental
Human,Soldier
Fungus,Elemental
nan
Horror
nan
nan
Goblin
Goblin,Rogue
nan
Wurm
nan
nan
nan
Human,Soldier
Horror
Minotaur,Shaman
nan
nan
nan
Construct
Aura
Hound
nan
Spirit
Elemental
Elf,Druid
Elf
nan
Aura
nan
Fish
Human,Warrior
Human,Soldier
Merfolk
nan
Spirit
Griffin
Horror
Ooze
Aura
nan
Atog
Human,Cleric
Wraith
Snake
Orc
nan
Human
nan
Illusion,Warrior
Aura
nan
nan
Rat
Treefolk
nan
Unicorn
Bir

### Preprocess prices

In [29]:
# prices.describe()
# prices.head()

In [30]:
prices_filtered = data_merged['price']

In [31]:
prices_np = prices_filtered.to_numpy()

### train, test split (90, 10)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(padded, prices_np, test_size=0.1, random_state=42)

### Standarize

In [33]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_test = y_scaler.transform(y_test.reshape(-1, 1))

## Model

In [43]:
input_text_model = tf.keras.layers.Input(shape=(max_length,))
text_model = tf.keras.layers.Embedding(vocab_size, 128, input_length=max_length)(input_text_model)
text_model = tf.keras.layers.LSTM(128, return_sequences=True, activation = LeakyReLU(alpha=0.1))(text_model)
text_model = tf.keras.layers.Dropout(0.5)(text_model)
# text_model = tf.keras.layers.LSTM(128, return_sequences=True, activation = LeakyReLU(alpha=0.1))(text_model)
# text_model = tf.keras.layers.Dropout(0.4)(text_model)
text_model = tf.keras.layers.Flatten()(text_model)

input_non_text_model = tf.keras.layers.Input(shape=(max_nontext_length, ))
non_text_model = tf.keras.layers.Dense(256, activation = LeakyReLU(alpha=0.1))(input_non_text_model)
non_text_model = tf.keras.layers.Dense(128, activation = LeakyReLU(alpha=0.1))(input_non_text_model)

concatenate_model = tf.keras.layers.Concatenate()([text_model, non_text_model])
concatenate_model = tf.keras.layers.Dense(512, activation = LeakyReLU(alpha=0.1))(concatenate_model)
# concatenate_model = tf.keras.layers.Dropout(0.4)(concatenate_model)
concatenate_model = tf.keras.layers.Dense(256, activation = LeakyReLU(alpha=0.1))(concatenate_model)
# concatenate_model = tf.keras.layers.Dropout(0.3)(concatenate_model)
# concatenate_model = tf.keras.layers.Dense(256, activation = LeakyReLU(alpha=0.1))(concatenate_model)
concatenate_model = tf.keras.layers.Dense(1)(concatenate_model)

model = Model([input_text_model, input_non_text_model], concatenate_model)

### Tensorboard

In [44]:
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

In [45]:
# optimizer = tf.keras.optimizers.RMSprop(0.0005) #todo: replace with Adam
optimizer = tf.keras.optimizers.Adam(0.00001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

In [46]:
X_train[0].shape,max_length, vocab_size

((344,), 287, 4096)

In [47]:
# X_train = np.zeros(X_train.shape)

### separate text and nontext data

In [48]:
X_train_text, X_train_nontext = X_train[:, 0:-max_nontext_length], X_train[:, -max_nontext_length:]

In [49]:
X_train_text.shape, X_train_nontext.shape, max_nontext_length

((17656, 287), (17656, 57), 57)

In [50]:
y_train

array([[-0.34682969],
       [-0.37774413],
       [-0.24378155],
       ...,
       [-0.38804894],
       [-0.37774413],
       [ 0.7351757 ]])

### Train

In [ ]:
epochs = 500
batch_size = 64
model.fit([X_train_text, X_train_nontext], y_train, epochs=epochs, callbacks=[tensorboard_callback], batch_size=batch_size, validation_split=0.2, shuffle=False)

Train on 14124 samples, validate on 3532 samples
Epoch 1/500
14124/14124 [==============================] - 103s 7ms/sample - loss: 0.9273 - mae: 0.5145 - mse: 0.9273 - val_loss: 0.9800 - val_mae: 0.4985 - val_mse: 0.9800
Epoch 2/500
14124/14124 [==============================] - 96s 7ms/sample - loss: 0.8772 - mae: 0.4904 - mse: 0.8772 - val_loss: 0.9395 - val_mae: 0.4908 - val_mse: 0.9395
Epoch 3/500
14124/14124 [==============================] - 99s 7ms/sample - loss: 0.8513 - mae: 0.4766 - mse: 0.8513 - val_loss: 0.9188 - val_mae: 0.4781 - val_mse: 0.9188
Epoch 4/500
14124/14124 [==============================] - 110s 8ms/sample - loss: 0.8372 - mae: 0.4682 - mse: 0.8372 - val_loss: 0.9086 - val_mae: 0.4670 - val_mse: 0.9086
Epoch 5/500
 1728/14124 [==>...........................] - ETA: 1:16 - loss: 0.8569 - mae: 0.4817 - mse: 0.8569

### read some value

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
def decode_sentence(text):
    text = scaler.inverse_transform(text)
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

def decode_price(val):
    return y_scaler.inverse_transform(val)

In [ ]:
for i in range(100):
    price = decode_price(y_train[i])
    if price > 1:
        print(i)
        print(decode_price(y_train[i]))

In [ ]:
print(decode_sentence(X_train[70]))

In [ ]:
data_merged[data_merged['name'].str.contains("Zur the Enchanter")]

In [ ]:
np.argwhere(np.isnan(padded))